In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
from pymongo import MongoClient

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "buster25"
HOST = 'nv-desktop-services.apporto.com'
PORT = 32896
DB = 'AAC'
COL = 'animals'

# Connect to database via CRUD Module
db = AnimalShelter(username, password, HOST, PORT, DB, COL)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#HTML image tag in the line below into the app.layout code according to your design
#unique identifier such as your name or date
app.layout = html.Div([
    html.Center(html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())))),
    html.Center(html.B(html.H5("Avery Regier SNHU CS 340 MongoDB Authentication"))),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
    #Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        # Set radio items for the filter query. Label is the clickable icon, value sets for
        # code in following sections for mongo queries
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water_rescue'},
                {'label': 'Mountain Wilderness Rescue', 'value': 'mountain_wilderness_rescue'},
                {'label': 'Disaster Individual Rescue', 'value': 'disaster_individual_rescue'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block'}
        )
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # add in some items
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        column_selectable="single",
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
        style={'display' : 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
            ),
            html.Div(
                id='map-id',
                className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################   

# add columns to read tables better
@app.callback(
        [Output('datatable-id', 'data'),
        Output('datatable-id', 'columns')],
        [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # from the radio, we can query individual data.
    df = {}
    if filter_type == 'water_rescue':
        df = {'animal_type': 'Dog', 'breed': {"$in": ['Labrador Retriever Mix',
                                                    'Chesapeake Bay Retriever',
                                                    'Newfoundland']},
                                                'sex_upon_outcome': 'Intact Female',
                                                'age_upon_outcome_in_weeks':
                                                {"$gte": 26.0, "$lte": 156.0}}
        
    elif filter_type == 'mountain_wilderness_rescue':
        df = {'animal_type': 'Dog','breed': {"$in": ['German Shepherd',
                                                    'Alaskan Malamute',
                                                    'Old English Sheepdog',
                                                    'Siberian Husky',
                                                     'Rottweiler']},
                                                'sex_upon_outcome': 'Intact Male',
                                                'age_upon_outcome_in_weeks':
                                                {"$gte": 26.0, "$lte": 156.0}}
        
    elif filter_type == 'disaster_individual_rescue':
        df = {'animal_type': 'Dog', 'breed': {"$in": ['Doberman Pinscher',
                                                    'German Shepherd',
                                                    'Golden Retriever',
                                                    'Bloodhound',
                                                    'Rottweiler']},
                                                'sex_upon_outcome': 'Intact Male',
                                                'age_upon_outcome_in_weeks':
                                                {"$gte": 20.0, "$lte": 300.0}}
        
    else:
        df = {}
    # we can read the data, then drop id if there is one, otherwise an error occurs, then we can return
    df = pd.DataFrame(db.read(df))
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records'), [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(         
        # we can unblur the commented code since it works well
           figure = px.pie(dff, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:27215/
